In [1]:
import argparse
import math
import time
import dill as pickle
from tqdm import tqdm
import numpy as np
import random
import os

import torch
import torch.nn.functional as F
import torch.optim as optim
from torchtext.data import Field, Dataset, BucketIterator
from torchtext.datasets import TranslationDataset

# import transformer.Constants as Constants
# from transformer.Models import Transformer, VAETransformer
# from transformer.Optim import ScheduledOptim


In [ ]:
batch_size = opt.batch_size
data = pickle.load(open(opt.data_pkl, 'rb'))

opt.max_token_seq_len = data['settings'].max_len
opt.src_pad_idx = data['vocab']['src'].vocab.stoi[Constants.PAD_WORD]
opt.trg_pad_idx = data['vocab']['trg'].vocab.stoi[Constants.PAD_WORD]

opt.src_vocab_size = len(data['vocab']['src'].vocab)
opt.trg_vocab_size = len(data['vocab']['trg'].vocab)

#========= Preparing Model =========#
if opt.embs_share_weight:
    assert data['vocab']['src'].vocab.stoi == data['vocab']['trg'].vocab.stoi, \
        'To sharing word embedding the src/trg word2idx table shall be the same.'

fields = {'src': data['vocab']['src'], 'trg':data['vocab']['trg']}

train = Dataset(examples=data['train'], fields=fields)
val = Dataset(examples=data['valid'], fields=fields)

train_iterator = BucketIterator(train, batch_size=batch_size, device=device, train=True)
val_iterator = BucketIterator(val, batch_size=batch_size, device=device)

In [2]:
data = pickle.load(open('../.data/pkl/aihub_mecab_test_1500.pkl', 'rb'))
print("data.keys()", data.keys())

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [19]:
max_token_seq_len = data['settings'].max_len
src_pad_idx = data['vocab']['src'].vocab.stoi['[pad]']
trg_pad_idx = data['vocab']['trg'].vocab.stoi['[pad]']
src_pad_idx

0

In [22]:
src_vocab_size = len(data['vocab']['src'].vocab)
trg_vocab_size = len(data['vocab']['trg'].vocab)
src_vocab_size, trg_vocab_size

(4388, 16231)

In [23]:
fields = {'src': data['vocab']['src'], 'trg':data['vocab']['trg']}

In [25]:
train = Dataset(examples=data['train'], fields=fields)
val = Dataset(examples=data['valid'], fields=fields)

In [28]:
train.examples[1].src

['[bos]',
 '▁even',
 '▁some',
 '▁problems',
 '▁can',
 '▁be',
 '▁predicted',
 '.',
 '[eos]']

In [29]:
train.examples[1].trg

['[bos]',
 '▁심지어',
 '▁어떤',
 '▁문제가',
 '▁발생할',
 '▁건',
 '지도',
 '▁어느',
 '▁정도',
 '▁예측',
 '이',
 '▁가능하다',
 '.',
 '[eos]']

In [36]:
(data['vocab']['src'].vocab)

4388